Kawasaki Quantum Summer Camp 2025

# 量子化学シミュレーション：サンプルベースの量子対角化

Kifumi Numata, IBM Quantum (Jul 31, 2025)

In [ ]:
# Import common packages first
import numpy as np
from math import comb
import warnings
import pyscf
import matplotlib.pyplot as plt
import pickle
from functools import partial

# Import qiskit classes
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_gate_map
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

# Import qiskit ecosystems
import ffsim
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_runtime import SamplerOptions

In [ ]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

**目次**

- 1: チュートリアル -  $N_2$ 分子の基底エネルギーを近似する手法を学ぶ
- 2: 演習 - パラメーターを調整して、基底エネルギーの改善をする

# 1: チュートリアル

## Step 1. 問題を量子回路にマッピングする

まず、$N_2$ 分子とそのプロパティを指定します。次に、量子回路を作成し、量子コンピューターから基底状態エネルギー推定用のサンプルを生成します。

<div class="alert alert-block alert-info">
    
⚠️ **Note:** 回路データは事前に計算されてこのノートブックで提供されているため、`pyscf` の実行の必要はありません。下のセルを実行して、**「最適化された量子回路をロードする」** セクションに直接進むことができます。
</div>


In [ ]:
import warnings, pickle
import numpy as np
warnings.filterwarnings("ignore")

# from pyscf import ao2mo, tools

# 分子特性の指定
num_orbitals = 16
num_elec_a = num_elec_b = 5
open_shell = False
spin_sq = 0
nelec = (num_elec_a, num_elec_b)

# pyscfを使用する場合は、ディスクから分子を読み込む
# mf_as = tools.fcidump.to_scf("utils/n2_fci.txt")
# hcore = mf_as.get_hcore()
# eri = ao2mo.restore(1, mf_as._eri, num_orbitals)
# nuclear_repulsion_energy = mf_as.mol.energy_nuc()

# pyscfを使用しない場合は、値をロードする
hcore = np.load("utils/hcore.npy") # 1電子ハミルトニアン
eri = np.load("utils/eri.npy") # 2電子ハミルトニアン
with open("utils/nuclear_repulsion_energy.pkl", "rb") as f:
    nuclear_repulsion_energy = pickle.load(f) # 原子核エネルギー

## Step 2. 最適化されたアンサッツ (仮の量子回路) をロードする

このチュートリアルでは、事前に準備された **最適化されたアンサッツ (仮の量子回路)ファイル** から始めます。 アンサッツは、$N_2$ 分子のハミルトニアンをもとに近似的に作成されています。またこのアンサッツは、IBM のヘビーヘックストポロジーを念頭に置いて作成されており、量子ビットの相互作用に **ジグザグパターン** を採用しています。 このパターンでは、同じ電子スピンの軌道がヘビーヘックスの格子全体にジグザグ線で接続されています (下図参照)。

![lucj_ansatz](https://quantum.cloud.ibm.com/assets-docs-learning/_next/image?url=%2Fdocs%2Fimages%2Ftutorials%2Fimproving-energy-estimation-of-a-fermionic-hamiltonian-with-sqd%2F7e0ee7e1-2d24-417f-ac59-25c58db79aa9.avif&w=1920&q=75)

In [ ]:
from qiskit import qpy
 
with open('utils/N2_UCJ_Transpiled.qpy', 'rb') as fd:
    isa_circuit = qpy.load(fd)[0]

## Step 3. 実験を実行する

ハードウェア実行用に回路を最適化したら、ターゲット ハードウェア上で回路を実行し、基底状態のエネルギー推定のためのサンプルを収集する準備が整います。

<div class="alert alert-block alert-warning">
    
⚠️ **Note:** QPU 上で回路を実行するためのコードはコメントアウトして、ユーザーの参照用に残してあります。このウォークスルーでは、実際のハードウェア上で実行するのではなく、以前に ``ibm_sherbrooke`` から取得した 10 万個のサンプルを読み込みます。

</div>


In [ ]:
import numpy as np

# from qiskit_ibm_runtime import SamplerV2 as Sampler

# sampler = Sampler(mode=backend)
# job = sampler.run([isa_circuit], shots=100_000)
# primitive_result = job.result()
# pub_result = primitive_result[0]
# counts = pub_result.data.meas.get_counts()

bit_array = np.load('utils/N2_device_bitarray.npy', allow_pickle=True).item()

## Step 4. SQD を使用した結果の後処理

このセクションでは、アンサッツから生成された量子サンプルを後処理して、分子の基底状態エネルギーを推定します。ここでは、データサンプルを修正することを繰り返し、精度を向上させます。

この手法ではユーザーが制御できる重要なオプションがいくつかあります。

* `max_iterations`: ループの反復回数。
* `num_batches`: 取り出すバッチの数 
* `samples_per_batch`: 各バッチに含めるデータ数
* `max_cycles`: 固有状態ソルバーの最大サイクル数


In [ ]:
%%time
# SQDのオプション
energy_tol = 1e-3  
occupancies_tol = 1e-3 
max_iterations = 5

# 固有状態ソルバーのオプション
num_batches = 5
samples_per_batch = 50
symmetrize_spin = True 
carryover_threshold = 1e-4 
max_cycle = 200
rng = np.random.default_rng(24)

# 中間結果を保存するためのリスト
result_history = [] 

def callback(results: list[SCIResult]): 
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")

result = diagonalize_fermionic_hamiltonian(
    hcore,
    eri,
    bit_array,
    samples_per_batch=samples_per_batch,
    norb=num_orbitals,
    nelec=nelec,
    num_batches=num_batches,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=max_iterations,
    symmetrize_spin=symmetrize_spin,
    carryover_threshold=carryover_threshold,
    callback=callback,
    seed=rng,
)

### 結果を視覚化する


In [ ]:
def plot_energy_and_occupancy(result_history):

    # Data for energies plot
    exact_energy = -109.10288938
    x1 = range(len(result_history))
    min_e = [
        min(result, key=lambda res: res.energy).energy + nuclear_repulsion_energy
        for result in result_history
    ]
    e_diff = [abs(e - exact_energy) for e in min_e]
    yt1 = [1.0, 1e-1, 1e-2, 1e-3, 1e-4]
    
    # Chemical accuracy (+/- 1 milli-Hartree)
    chem_accuracy = 0.001
    
    # Data for avg spatial orbital occupancy
    y2 = np.sum(result.orbital_occupancies, axis=0)
    x2 = range(len(y2))
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot energies
    axs[0].plot(x1, e_diff, label="energy error", marker="o")
    axs[0].set_xticks(x1)
    axs[0].set_xticklabels(x1)
    axs[0].set_yticks(yt1)
    axs[0].set_yticklabels(yt1)
    axs[0].set_yscale("log")
    axs[0].set_ylim(1e-4)
    axs[0].axhline(y=chem_accuracy, color="#BF5700", linestyle="--", label="chemical accuracy")
    axs[0].set_title("Approximated Ground State Energy Error vs SQD Iterations")
    axs[0].set_xlabel("Iteration Index", fontdict={"fontsize": 12})
    axs[0].set_ylabel("Energy Error (Ha)", fontdict={"fontsize": 12})
    axs[0].legend()
    
    # Plot orbital occupancy
    axs[1].bar(x2, y2, width=0.8)
    axs[1].set_xticks(x2)
    axs[1].set_xticklabels(x2)
    axs[1].set_title("Avg Occupancy per Spatial Orbital")
    axs[1].set_xlabel("Orbital Index", fontdict={"fontsize": 12})
    axs[1].set_ylabel("Avg Occupancy", fontdict={"fontsize": 12})
    
    print(f"Exact energy: {exact_energy:.5f} Ha")
    print(f"SQD energy: {min_e[-1]:.5f} Ha")
    print(f"Absolute error: {e_diff[-1]:.5f} Ha")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_energy_and_occupancy(result_history)

最初のプロットは、私たちの近似値が化学的精度からどの程度離れているかを示しています。（一般的には ``1 kcal/mol`` $\approx$ ``1.6 mH``).

2 番目のプロットは、最終反復後の各空間軌道の平均占有率を示しています。私たちのソリューションでは、スピンアップ電子とスピンダウン電子の両方が最初の 5 つの軌道を高い確率で占有していることがわかります。

# 2. 演習 

エネルギー推定の精度を向上させるという課題に取り組みます。Part 1  の初期設定ではベースラインの近似値を提供しましたが、ここでは、さまざまなパラメータの選択がパフォーマンスにどのように影響するかを探ります。

<a id="exercise_1"></a>
<div class="alert alert-block alert-success">
    
<b>基底状態エネルギー推定の改良</b> 

チュートリアルの例では、小さなサブスペースを使用し、反復ごとにバッチ数を 5 に制限して、分子の基底状態エネルギーの初期近似値を取得しました。ただし、精度を向上させる余地は大きくあります。**Part 2**のタスクは、次の2つの主要なパラメーターを試して、エネルギー推定値を向上させることです。

- **サブスペース サイズ** (`samples_per_batch`): バッチあたりのサンプル数を調整して、ハミルトニアンの射影と対角化に使用するサブスペース サイズを定義します。この値を大きくすると精度が向上しますが、計算負荷が増加する可能性があります。
- **バッチ数** (`num_batches`): バッチ数を変更して、各反復での計算効率と結果の精度のバランスを見つけます。

これらのパラメータを戦略的に使用して、推定された基底状態エネルギーを実際の値にできるだけ近づけることを目指します。ノイズやハードウェア制約の影響を考慮しながら、さまざまな組み合わせをテストしてエラーを最小限に抑えます。

頑張ってください。真の基底状態エネルギーにどれだけ近づけるか見てみましょう。


</div>

まず、上記と同じループを実行し、``num_batches`` および ``samples_per_batch`` 引数を入力として受け取る関数を定義します。

In [ ]:
def sqd_configuration_recovery(num_batches: int, samples_per_batch: int) -> float:
    # SQDのオプション
    energy_tol = 1e-3  
    occupancies_tol = 1e-3 
    max_iterations = 5
    
    # 固有状態ソルバーのオプション
    symmetrize_spin = True 
    carryover_threshold = 1e-4 
    max_cycle = 200
    rng = np.random.default_rng(24)
    
    result = diagonalize_fermionic_hamiltonian(
        hcore,
        eri,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=num_orbitals,
        nelec=nelec,
        num_batches=num_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        callback=callback,
        seed=rng,
    )
    return result_history

In [ ]:
### この下にコードを記入してください ###

num_batches = 
samples_per_batch = 

### この行以降のコードは変更しないでください ###

In [ ]:
%%time
result_history = [] 
energy_hist = sqd_configuration_recovery(num_batches, samples_per_batch)

In [ ]:
plot_energy_and_occupancy(result_history)

1章のチュートリアルの結果は以下でした。

    Exact energy: -109.10289 Ha
    SQD energy: -109.02333 Ha
    Absolute error: 0.07956 Ha

あなたの演習の結果は、より良くなりましたか？